# A Simple Convolutional Neural Network
![gChvH4](https://gitee.com/pxqp9W/testmarkdown/raw/master/imgs/2020/07/gChvH4.png)
![VBcUKI](https://gitee.com/pxqp9W/testmarkdown/raw/master/imgs/2020/07/VBcUKI.png)

In [1]:
import torch 
from torchvision import transforms 
from torchvision import datasets 
from torch.utils.data import DataLoader 
import torch.nn.functional as F 
import torch.optim as optim
import time

## 1、Prepare Dataset

In [2]:
batch_size = 64

## Convert the PIL Image to Tensor.
transform = transforms.Compose([
    transforms.ToTensor(), # 图像转变为ToTensor，取值变为0-1
    transforms.Normalize((0.1307, ), (0.3081, )) 
    # 0.1307是均值mean, 0.3081是标准差std
    # 这两个数是根据 MNIST数据集的特征得到的经验值
])

## 构造loader
train_dataset = datasets.MNIST(root='./dataset/mnist/', 
                               train=True, 
                               download=True, 
                               transform=transform) # transform放入到了数据集中，这样每次取出来的数据都会直接做变换
train_loader = DataLoader(train_dataset, 
                          shuffle=True, 
                          batch_size=batch_size)
test_dataset = datasets.MNIST(root='./dataset/mnist/' , 
                              train=False, 
                              download=True, 
                              transform=transform)
test_loader = DataLoader(test_dataset, 
                         shuffle=False, 
                         batch_size=batch_size)

## 2、Design CNN Model 

In [3]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__() 
        self.conv1 = torch.nn.Conv2d(1, 10, kernel_size=5) 
        self.conv2 = torch.nn.Conv2d(10, 20, kernel_size=5) 
        self.pooling = torch.nn.MaxPool2d(2) 
        self.fc = torch.nn.Linear(320, 10)
    
    def forward(self, x):
        # Flatten data from (n, 1, 28, 28) to (n, 784)
        batch_size = x.size(0) 
        x = F.relu(self.pooling(self.conv1(x))) 
        x = F.relu(self.pooling(self.conv2(x)))
        x = x.view(batch_size, -1) # flatten 
        x = self.fc(x)  # 因为后面要用PyTorch的交叉熵函数，因此最后一层不做激活
        return x
    
model = Net()

## 3、Construct Loss and Optimizer

In [4]:
criterion = torch.nn.CrossEntropyLoss() 
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5) # momentum表示动量

## 4、Train & Test

In [5]:
## 将训练过程封装到函数
def train(epoch):
    running_loss = 0.0
    for batch_idx, data in enumerate(train_loader, 0):
        inputs, target = data 
        optimizer.zero_grad()
        
        # forward + backward + update
        outputs = model(inputs) 
        loss = criterion(outputs, target) 
        loss.backward() 
        optimizer.step()
        running_loss += loss.item() 
        
        if batch_idx % 300 == 299: 
            print('[%d, %5d] loss: %.3f' % (epoch + 1, batch_idx + 1, running_loss / 300)) 
            running_loss = 0.0

In [6]:
## 将测试过程也封装为函数
def test():
    correct = 0 
    total = 0
    with torch.no_grad(): # Test过程中不需要计算梯度，可以使用“with torch.no_grad()”
        for data in test_loader:
            images, labels = data 
            outputs = model(images) 
            _, predicted = torch.max(outputs.data, dim=1) # torch.max用来找行中的最大值所在下标 dim1表示沿着第1个维度找（行是第0个维度，列是第1个维度）
            total += labels.size(0) # labels是一个N*1的向量，取size()得到的是元组(N,1),size(0)返回的是N
            correct += (predicted == labels).sum().item() # 预测正确的数量，predicted == labels进行的是张量之间的比较运算
    print('Accuracy on test set: %d %%' % (100 * correct / total))

## Main Function

In [7]:
if __name__ == '__main__':
    start = time.time()
    for epoch in range(10):
        train(epoch) 
        test()
    end = time.time()
    print("Total time: ", end - start , "s")

[1,   300] loss: 0.639
[1,   600] loss: 0.194
[1,   900] loss: 0.141
Accuracy on test set: 96 %
[2,   300] loss: 0.106
[2,   600] loss: 0.099
[2,   900] loss: 0.094
Accuracy on test set: 98 %
[3,   300] loss: 0.077
[3,   600] loss: 0.075
[3,   900] loss: 0.075
Accuracy on test set: 98 %
[4,   300] loss: 0.061
[4,   600] loss: 0.063
[4,   900] loss: 0.065
Accuracy on test set: 98 %
[5,   300] loss: 0.052
[5,   600] loss: 0.056
[5,   900] loss: 0.060
Accuracy on test set: 98 %
[6,   300] loss: 0.049
[6,   600] loss: 0.050
[6,   900] loss: 0.049
Accuracy on test set: 98 %
[7,   300] loss: 0.048
[7,   600] loss: 0.042
[7,   900] loss: 0.047
Accuracy on test set: 98 %
[8,   300] loss: 0.043
[8,   600] loss: 0.046
[8,   900] loss: 0.040
Accuracy on test set: 98 %
[9,   300] loss: 0.036
[9,   600] loss: 0.041
[9,   900] loss: 0.041
Accuracy on test set: 98 %
[10,   300] loss: 0.038
[10,   600] loss: 0.038
[10,   900] loss: 0.036
Accuracy on test set: 98 %
Total time:  119.91048622131348 s
